In [48]:
import pandas as pd
import re
import json
import os
from tqdm import tqdm

In [2]:
files_to_dowload = ["GB_youtube_trending_data", "US_youtube_trending_data"]

df_positive = pd.concat([pd.read_csv(f"data/{name}.csv") for name in files_to_dowload])
df_positive = df_positive.sort_values(by=['view_count'], ascending=False)
df_positive = df_positive.drop_duplicates(subset=['video_id'])
df_positive = df_positive[['video_id', 'title', 'publishedAt', 'channelId', 'channelTitle', 'view_count', 'likes', 'comment_count', 'thumbnail_link', 'description']]

df_positive["target"] = 1
df_positive

,video_id,title,publishedAt,channelId,channelTitle,view_count,likes,comment_count,thumbnail_link,description,target
56374,WMweEpGlu_U,BTS (방탄소년단) 'Butter' Official MV,2021-05-21T03:46:13Z,UC3IZKseVpdzPSBaWxBxundA,HYBE LABELS,264407389,16021534,6738537,https://i.ytimg.com/vi/WMweEpGlu_U/default.jpg,BTS (방탄소년단) 'Butter' Official MV Credits: Dire...,1
3358,gdZLi9oWNZg,BTS (방탄소년단) 'Dynamite' Official MV,2020-08-21T03:58:10Z,UC3IZKseVpdzPSBaWxBxundA,Big Hit Labels,232649205,15735533,6065230,https://i.ytimg.com/vi/gdZLi9oWNZg/default.jpg,BTS (방탄소년단) 'Dynamite' Official MVCredits:Dire...,1
73564,hdmx71UjBXs,Turn into orbeez - Tutorial #Shorts,2021-07-03T04:04:57Z,UCt8z2S30Wl-GQEluFVM8NUw,FFUNTV,206202284,6840430,2826,https://i.ytimg.com/vi/hdmx71UjBXs/default.jpg,Turn into orbeez - Tutorial #ShortsHey guys! W...,1
68781,qF0N19MgI3Q,JETSKI WAX PRANK!! 🤣 - #Shorts,2021-07-04T19:00:09Z,UCCNaMMlI3cOc7yFg52riTqg,Julius Dein,194625542,4935519,26279,https://i.ytimg.com/vi/qF0N19MgI3Q/default.jpg,My name is Julius Dein : ) I'm a 27 year old m...,1
4980,vRXZj0DzXIA,BLACKPINK - 'Ice Cream (with Selena Gomez)' M/V,2020-08-28T04:00:11Z,UCOmHUn--16B90oW2L6FRR3A,BLACKPINK,184778248,11795670,2735997,https://i.ytimg.com/vi/vRXZj0DzXIA/default.jpg,BLACKPINK - ‘Ice Cream (with Selena Gomez)’Com...,1
...,...,...,...,...,...,...,...,...,...,...,...
19027,AWXvClaRtsI,Celebrating Maria Tallchief,2020-11-14T22:39:13Z,UCdq61m8s_48EhJ5OM_MCeGw,GoogleDoodles,0,0,0,https://i.ytimg.com/vi/AWXvClaRtsI/default.jpg,In honor of Native American Heritage Month in ...,1
65535,Fn7R04uYG1E,Celebrate Pride loud and proud with Olly Alexa...,2021-06-25T20:06:45Z,UC9MNPh0ZtaDW0t-v7DlOEQA,Years & Years,0,0,278,https://i.ytimg.com/vi/Fn7R04uYG1E/default.jpg,Olly Alexander and Mawaan Rizwan are celebrati...,1
65049,kmk5vciFbek,Demi Lovato performs their greatest hits this ...,2021-06-26T00:07:19Z,UCZkURf9tDolFOeuw_4RD7XQ,Demi Lovato,0,0,138,https://i.ytimg.com/vi/kmk5vciFbek/default.jpg,"On June 25, celebrate Pride 2021 with Demi Lov...",1
74492,Hb3rmh-_FMw,Introducing the shorter side of YouTube,2021-08-10T15:04:25Z,UCBR8-60-B28hp2BmDPdntcQ,YouTube,0,22030,0,https://i.ytimg.com/vi/Hb3rmh-_FMw/default.jpg,*****EPILEPSY WARNING ********Watch and create...,1


In [5]:
df_negat_michal = pd.read_csv("data/channel_related_related_last_1541.csv", index_col=0)

df_negat_michal = df_negat_michal[['video_id', 'title', 'publishedAt', 'channelId', 'channelTitle', 'view_count', 'likes', 'comment_count', 'thumbnail_link', 'description']]
df_negat_michal["target"] = 0
df_negat_michal

,video_id,title,publishedAt,channelId,channelTitle,view_count,likes,comment_count,thumbnail_link,description,target
0,iRC8sQOVcO8,How To Win Salam Kosh All Matches in One Day M...,2020-11-27T15:30:07Z,UCG8xgfCmEOqvYC7vHgwzOeg,Malh TV HD,4807.0,84.0,3.0,https://i.ytimg.com/vi/iRC8sQOVcO8/default.jpg,FightingGame #Wrestling #Malakhra How To Win S...,0
1,GT1ikFQJAC8,Teddy Sheedi is On Fire | Teddy Sheedi New Mal...,2020-11-20T15:30:01Z,UCG8xgfCmEOqvYC7vHgwzOeg,Malh TV HD,4968.0,76.0,6.0,https://i.ytimg.com/vi/GT1ikFQJAC8/default.jpg,Teddy Sheedi is On Fire | Teddy Sheedi New Mal...,0
2,BYmP6__6b9I,Traditional Game Malakhra Kushti Ghulam Hussai...,2021-03-15T17:20:10Z,UCG8xgfCmEOqvYC7vHgwzOeg,Malh TV HD,11908.0,146.0,5.0,https://i.ytimg.com/vi/BYmP6__6b9I/default.jpg,Traditional Game Malakhra Kushti Ghulam Hussai...,0
3,V9PDks2qd8Q,Sindhi Traditional Game Malakhra Kushti Ubedul...,2021-04-15T15:38:20Z,UCG8xgfCmEOqvYC7vHgwzOeg,Malh TV HD,112032.0,878.0,14.0,https://i.ytimg.com/vi/V9PDks2qd8Q/default.jpg,Sindhi Traditional Game Malakhra Kushti Ubedul...,0
4,ZBQwuIXgZ_8,Traditional Game Malakhra Kushti Pehlwan Ghula...,2021-03-08T17:52:10Z,UCG8xgfCmEOqvYC7vHgwzOeg,Malh TV HD,9232.0,112.0,4.0,https://i.ytimg.com/vi/ZBQwuIXgZ_8/default.jpg,Traditional Game Malakhra Kushti Pehlwan Ghula...,0
...,...,...,...,...,...,...,...,...,...,...,...
7593,GQuJ2uirMxI,WHO SENDS MORE MEMES? Best SAMBA dancer? | Rod...,2021-06-15T12:00:06Z,UCWV3obpZVGgJ3j9FVhEjF2Q,Real Madrid,419233.0,28879.0,615.0,https://i.ytimg.com/vi/GQuJ2uirMxI/default.jpg,Check out the second episode of our Teammates ...,0
7594,kTK-LgsGK9o,VARANE says GOODBYE to Real Madrid teammates,2021-07-30T13:15:03Z,UCWV3obpZVGgJ3j9FVhEjF2Q,Real Madrid,1336096.0,53910.0,3153.0,https://i.ytimg.com/vi/kTK-LgsGK9o/default.jpg,Raphäel Varane says goodbye to Real Madrid aft...,0
7595,epef95O-EwY,The AMAZING pitch removal system at the new Sa...,2021-11-04T15:53:34Z,UCWV3obpZVGgJ3j9FVhEjF2Q,Real Madrid,1389960.0,51426.0,2389.0,https://i.ytimg.com/vi/epef95O-EwY/default.jpg,Check out this amazing new rendered footage of...,0
7596,pog89yj0SM4,SPECTACULAR 2 vs 2 in REAL MADRID training!,2021-08-25T17:20:57Z,UCWV3obpZVGgJ3j9FVhEjF2Q,Real Madrid,1346029.0,25434.0,886.0,https://i.ytimg.com/vi/pog89yj0SM4/default.jpg,"Join the likes of Gareth Bale, David Alaba and...",0


In [11]:
files = list(filter(lambda x: re.search(r".json", x), os.listdir("data/downloaded_videos_patryk")))
all_data = []

for file_name in files:
    with open(f"data/downloaded_videos_patryk/{file_name}") as file:
        data = json.load(file)
        all_data.append(data)

df_patryk = pd.DataFrame(all_data, columns = ['video_id', 'title', 'publishedAt', 'channelId', 'channelTitle', 'view_count', 'likes', 'comment_count', 'thumbnail_link', 'description'])
df_patryk["target"] = 0

df_patryk['view_count'] = df_patryk['view_count'].astype(float)
df_patryk['likes'] = df_patryk['likes'].astype(float)
df_patryk['comment_count'] = df_patryk['comment_count'].astype(float)
df_patryk

,video_id,title,publishedAt,channelId,channelTitle,view_count,likes,comment_count,thumbnail_link,description,target
0,---D0QE8hxo,Denis Villeneuve Opens Up About the Success of...,2021-10-30T00:30:00Z,UCZ8Sxmkweh65HetaZfR8YuA,The Hollywood Reporter,26734.0,521.0,83.0,https://i.ytimg.com/vi/---D0QE8hxo/default.jpg,The filmmaker reflects back on his journey and...,0
1,--DKkzWVh-E,Why Retaining Walls Collapse,2021-12-07T13:00:00Z,UCMOqf8ab-42UUQIdVoKwjlQ,Practical Engineering,1114808.0,41354.0,1305.0,https://i.ytimg.com/vi/--DKkzWVh-E/default.jpg,One of the most important (and innocuous) part...,0
2,--FmExEAsM8,IVE 아이브 &#39;ELEVEN&#39; MV,2021-12-01T09:00:03Z,UCYDmx2Sfpnaxg488yBpZIGg,starshipTV,41048214.0,1116817.0,64482.0,https://i.ytimg.com/vi/--FmExEAsM8/default.jpg,IVE Twitter : https://twitter.com/IVEstarship ...,0
3,--IC6l2-Hks,Minecraft Carnival Game...,2021-04-21T21:43:41Z,UCZWBGXa6kiaWo9-pE8CX6Dg,Dream Shorts,5223956.0,369916.0,10547.0,https://i.ytimg.com/vi/--IC6l2-Hks/default.jpg,Minecraft Carnival Game... #shorts @Technoblad...,0
4,--Rd2-yrb8s,Sean Penn Discusses President Trump&#39;s COVI...,2020-09-02T19:08:58Z,UCLKeNNZBAsSyN3n1SHYGDnw,CORE,4397.0,68.0,48.0,https://i.ytimg.com/vi/--Rd2-yrb8s/default.jpg,Community Organized Relief Effort Founder Sean...,0
...,...,...,...,...,...,...,...,...,...,...,...
23377,_ZtTi3ZDBuA,Hawaii Volcano Update: Magnitude 6.2 Earthquak...,2021-10-12T01:51:52Z,UCjB5Oj6z1g4Tr0dLmx3cdsA,Big Island Video News,111471.0,NaN,201.0,https://i.ytimg.com/vi/_ZtTi3ZDBuA/default.jpg,HAWAIʻI - Stunning video from above recently c...,0
23378,_zzEDrYTkkg,9/21/20,2020-09-21T16:21:18Z,UCaDQ-IrcLEcK6SnlsHdWw6Q,Demi Adejuyigbe,2156523.0,92840.0,3752.0,https://i.ytimg.com/vi/_zzEDrYTkkg/default.jpg,"sweaty mcmovesalot is back, in upscaled 720p d...",0
23379,__0JRXzt5mI,These Memes are FACTS Compilation part 8,2021-03-13T19:36:11Z,UCyQHab6hU_5cqhK6wRS173w,Bentellect,490105.0,19511.0,393.0,https://i.ytimg.com/vi/__0JRXzt5mI/default.jpg,My Social Media & FACTS Merch https://linktr.e...,0
23380,__JFeoChvu8,Shaq &amp; Chuck Roasting Zion Williamson on I...,2021-11-03T03:27:00Z,UCqQo7ewe87aYAe7ub5UqXMw,House of Highlights,4317259.0,67788.0,5787.0,https://i.ytimg.com/vi/__JFeoChvu8/default.jpg,SHOP OUR MERCH: https://hoh.world Follow our I...,0


In [12]:
df_all = pd.concat([df_positive, df_negat_michal, df_patryk])

# leave only trending videos
df_all = df_all.sort_values(by=['target'], ascending=False)
df_all = df_all.drop_duplicates(subset=['video_id'])
df_all

,video_id,title,publishedAt,channelId,channelTitle,view_count,likes,comment_count,thumbnail_link,description,target
56374,WMweEpGlu_U,BTS (방탄소년단) 'Butter' Official MV,2021-05-21T03:46:13Z,UC3IZKseVpdzPSBaWxBxundA,HYBE LABELS,264407389.0,16021534.0,6738537.0,https://i.ytimg.com/vi/WMweEpGlu_U/default.jpg,BTS (방탄소년단) 'Butter' Official MV Credits: Dire...,1
8755,IhBen8Gjtpc,How to add custom photo Widget on iOS 14,2020-09-19T18:40:48Z,UCrSvDunJEc1CME4-KvhW_3Q,Howfinity,535280.0,6530.0,377.0,https://i.ytimg.com/vi/IhBen8Gjtpc/default.jpg,How to add custom photo Widget on iOS 14With i...,1
51578,uKTy-JRhMik,My Little Brother Meets My Son.. *Emotional*,2021-04-24T15:30:05Z,UCSqEMt26wLsKC6K_wTyr9Cw,Capron Funk,535485.0,25577.0,736.0,https://i.ytimg.com/vi/uKTy-JRhMik/default.jpg,Corey Funk my little brother meets my son for ...,1
45262,zi7JYQiFiGQ,Serbia come from 2 goals down to rescue draw! ...,2021-03-27T22:28:32Z,UCNAf1k0yIjyGu3k9BwAg3lg,Sky Sports Football,535546.0,7770.0,1648.0,https://i.ytimg.com/vi/zi7JYQiFiGQ/default.jpg,SUBSCRIBE ► http://bit.ly/SSFootballSubPREMIER...,1
11989,fXysipmTxcQ,FASTEST TOP GEAR LAP? Ferrari SF90 Stiglap | T...,2020-10-06T16:15:47Z,UCjOl2AUblVmg2rA_cRgZkFg,Top Gear,535651.0,9525.0,881.0,https://i.ytimg.com/vi/fXysipmTxcQ/default.jpg,"The 986bhp Ferrari SF90 is, unsurprisingly, no...",1
...,...,...,...,...,...,...,...,...,...,...,...
2763,6RQ5fdLyLwc,15 Times Animals Proved Size Does Not Matter,2020-12-15T17:00:02Z,UCTDJ9osmaIty0MVkUZymVmQ,Amerikano,70823.0,1905.0,87.0,https://i.ytimg.com/vi/6RQ5fdLyLwc/default.jpg,We're all pretty familiar with the Biblical st...,0
2762,6RPtm5TeMR4,Dominique Moceanu offers insight into Simone B...,2021-07-28T05:14:38Z,UCNBmxc6FvKyxtCpUygcdINA,WKYC Channel 3,255595.0,2119.0,1147.0,https://i.ytimg.com/vi/6RPtm5TeMR4/default.jpg,"Like the rest of the world, Dominique Moceanu ...",0
2761,6rnhnfpLcJM,THESE ACTS OF KINDNESS ARE THE MOST HEARTWARMI...,2021-11-27T16:00:06Z,UC5OEQdhJUH_gK0231-f6LyA,Bobby Moore Facts,835027.0,87971.0,656.0,https://i.ytimg.com/vi/6rnhnfpLcJM/default.jpg,,0
2760,6riDJMI-Y8U,JUJUTSU KAISEN - Ending | Lost in Paradise fea...,2020-11-20T19:00:03Z,UC6pGDc4bFGD1_36IKv3FnYg,Crunchyroll Collection,17716475.0,446946.0,11305.0,https://i.ytimg.com/vi/6riDJMI-Y8U/default.jpg,Artist: ALI Song: Lost in Paradise feat. AKLO ...,0


In [ ]:
df_all.to_csv("data/merged_data_youtube.csv")

In [50]:
ids_to_left = df_all[df_all["target"] == 0]["video_id"]
ids_to_left

13076    mJc7WDPrTbk
13074    mJAjGNhYD9I
13075    mJBlgIA3K24
13079    mJdkwoBRe3Q
13077    mjCqI-AxdQE
            ...     
2763     6RQ5fdLyLwc
2762     6RPtm5TeMR4
2761     6rnhnfpLcJM
2760     6riDJMI-Y8U
23381    __uhg38kmWk
Name: video_id, Length: 25553, dtype: object

In [49]:
files = os.listdir("data/images/negative")
for item in tqdm(files):
    name = item.replace(".jpg", "")
    if name not in list(ids_to_left):
        os.remove(f"data/images/negative/{name}.jpg")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27045/27045 [01:14<00:00, 360.67it/s]


In [21]:




for file_name in files:
    os.rename(f"data/images/negative/{file_name}", f"data/images/negative/{file_name.replace('.png', '.jpg')}")
